# Core Model Guide: MuskingumChannelModel

This notebook provides a detailed guide to the `MuskingumChannelModel`, which is a widely used hydrological model for simulating the flow of water in a river channel or reach. Its primary use is for **flood routing**.

## 1. Theoretical Background

The Muskingum method is a simple and effective way to model how a flood wave changes shape as it moves downstream. It accounts for two key phenomena:
1.  **Translation**: The time it takes for the wave to travel from the upstream to the downstream end.
2.  **Attenuation**: The flattening and spreading out of the flood wave due to channel storage effects.

The core of the model is the Muskingum equation:

```
O_t = C1*I_t + C2*I_{t-1} + C3*O_{t-1}
```

Where:
- `O` is the Outflow and `I` is the Inflow.
- `t` is the current time step and `t-1` is the previous time step.
- `C1`, `C2`, `C3` are routing coefficients calculated from the physical characteristics of the channel, represented by the parameters `K` and `x`, and the simulation time step `dt`.

## 2. API and Parameters

Let's import the necessary libraries and the model.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.modeling.storage_models import MuskingumChannelModel

The `MuskingumChannelModel` is initialized with:

- `K` (float): The travel time constant for the reach (in hours). This represents the time it takes for the peak of the flood wave to travel through the reach.
- `x` (float): The weighting factor, dimensionless, between 0 and 0.5. It controls the amount of attenuation. A value of 0 means maximum attenuation (like a reservoir), while 0.5 means almost no attenuation.
- `dt` (float): The simulation time step (in hours). For the model to be stable, `dt` should be chosen carefully relative to `K`.
- `initial_inflow` (float): The inflow at the start of the simulation (m³/s).
- `initial_outflow` (float): The outflow at the start of the simulation (m³/s). Typically, these are equal for a steady-state start.

## 3. Code Example: Routing a Flood Wave

We will create a synthetic flood wave (a triangular inflow hydrograph) and route it through a channel reach to see how the model simulates translation and attenuation.

In [ ]:
# 1. Model Initialization
model = MuskingumChannelModel(
    K=20.0,            # 20-hour travel time
    x=0.2,             # A typical value for natural channels
    dt=1.0,            # 1-hour time step
    initial_inflow=10.0, # Starting with a baseflow of 10 m^3/s
    initial_outflow=10.0
)

# 2. Simulation Setup
# Create a triangular inflow hydrograph representing a storm event
base_flow = 10.0
peak_inflow = 100.0
time_to_peak = 24 # hours
recession_time = 48 # hours
total_duration = 100 # hours

inflow_hydrograph = np.full(total_duration, base_flow)
inflow_hydrograph[:time_to_peak] = np.linspace(base_flow, peak_inflow, time_to_peak)
inflow_hydrograph[time_to_peak:time_to_peak + recession_time] = np.linspace(peak_inflow, base_flow, recession_time)

history = {
    "time": [],
    "inflow": [],
    "outflow": []
}

# 3. Simulation Loop
for t in range(total_duration):
    # Get the inflow for the current time step from our hydrograph
    current_inflow = inflow_hydrograph[t]
    
    # Set the model's input
    model.input.inflow = current_inflow
    
    # Run the model for one step
    outflow = model.step()
    
    # Record results
    history["time"].append(t)
    history["inflow"].append(current_inflow)
    history["outflow"].append(outflow)

print("Simulation complete.")

## 4. Visualization

Plotting the inflow and outflow hydrographs on the same axes is the best way to visualize the model's effect.

In [ ]:
plt.figure(figsize=(12, 6))

plt.plot(history['time'], history['inflow'], label='Inflow Hydrograph', color='blue', linestyle='--')
plt.plot(history['time'], history['outflow'], label='Outflow Hydrograph', color='red', linewidth=2)

plt.title('Flood Wave Routing with Muskingum Model')
plt.xlabel('Time (hours)')
plt.ylabel('Flow Rate (m³/s)')
plt.legend()
plt.grid(True)
plt.show()

### Analysis of Results

The plot clearly shows the two main effects of channel routing:

1.  **Attenuation**: The peak of the outflow hydrograph (red line) is significantly lower than the peak of the inflow hydrograph (blue dashed line).
2.  **Translation (Lag)**: The peak of the outflow occurs later in time than the peak of the inflow. 

This demonstrates that the `MuskingumChannelModel` correctly simulates the fundamental behavior of open-channel flow during a flood event.